# Computing SHAP Values
For the purpose of testing, we have made available our training dataset, which includes both eyes-open and eyes-closed features for the 12 regional interpolated channels. The model parameters have been configured to match those of our top-performing model for this dataset.

In [ ]:
import pandas as pd
import shap
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.preprocessing import StandardScaler
import pickle

In [ ]:
def load_object(fname):
    try:
        with open(fname + ".pickle", "rb") as f:
            return pickle.load(f)
    except Exception as ex:
        print("Error during unpickling object (Possibly unsupported):", ex)


data = load_object('../../data/example_training_set/training_set')

In [ ]:
x = data['x']
groups = data['group']
y = data['y']
x_names = data['x_names']

print(f"Dataset contains {len(x)} Samples of size {len(x[0])}")

In [ ]:
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [ ]:
y_skf = [int(age) for age in y]
skf_vals = []
skf = StratifiedGroupKFold(n_splits=3, shuffle=True, random_state=126)
for fold, (train_index, test_index) in enumerate(skf.split(x, y_skf, groups)):
    skf_vals.append((train_index, test_index))

In [ ]:
model_param = {
    'n_neighbors': 21,
    'leaf_size': 98,
    'p': 1
}

In [ ]:
best_fold = 0
best_score = 5
best_model = None
for fold in range(len(skf_vals)):
    x_train = [x[i] for i in skf_vals[fold][0]]
    x_test = [x[i] for i in skf_vals[fold][1]]
    y_train = [y[i] for i in skf_vals[fold][0]]
    y_test = [y[i] for i in skf_vals[fold][1]]

    model = KNeighborsRegressor(**model_param, n_jobs=30)
    model.fit(x_train, y=y_train)

    preds = model.predict(x_test)
    mae = mean_absolute_error(y_test, preds)
    if mae < best_score:
        best_fold = fold
        best_score = mae
        best_model = model

In [ ]:
x_train = [x[i] for i in skf_vals[best_fold][0]]
x_test = [x[i] for i in skf_vals[best_fold][1]]
y_train = [y[i] for i in skf_vals[best_fold][0]]
y_test = [y[i] for i in skf_vals[best_fold][1]]

x_train_df = pd.DataFrame(x_train, columns=x_names)
x_test_df = pd.DataFrame(x_test, columns=x_names)

# Initialize the shap explainer
explainer = shap.KernelExplainer(best_model.predict, shap.sample(x_train_df, 10), num_jobs=-2)

# Compute Shap values for all instances in X_test
shap_values = explainer.shap_values(x_test_df)